In [ ]:
import pandas as pd
import numpy as np
import warnings

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix, precision_score

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    # path = '~/Documents/GitHub/f1-analytics/'
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [ ]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/merged.csv')
params = pd.read_csv(path+'parameters/random-forest-regression.csv')

In [ ]:
### Season to test results

N = 2021

In [ ]:
df = data.copy()
# df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season < N]

scaler = StandardScaler()
# X_train = train.drop(['podium'], axis=1)
X_train = train.drop(['driver_points_after_race'], axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.podium.values)

In [ ]:
def score_regression(model):
    correct_predictions = 0
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        # X_test = test.drop(['podium'], axis=1)
        X_test = test.drop(['driver_points_after_race'], axis=1)
        y_test = test.driver_points_after_race

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns = ['results'])
        prediction_df['driver_points_after_race'] = y_test.reset_index(drop=True)
        # prediction_df['podium'] = y_test.reset_index(drop=True)
        prediction_df['actual'] = prediction_df.driver_points_after_race.map(lambda x: 1 if x == 1 else 0)
        prediction_df.sort_values('results', ascending=False, inplace=True)
        prediction_df.reset_index(inplace=True, drop=True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        correct_predictions += precision_score(prediction_df.actual, prediction_df.predicted)

    return correct_predictions / df[df.season == N]['round'].nunique()

In [ ]:
# Random Forest Regressor


model_params = (criterion, max_features, max_depth)
model = RandomForestRegressor(criterion = criterion, max_features = max_features, max_depth = max_depth)
model.fit(X_train, y_train)

score = score_regression(model)
        